In [18]:
from netCDF4 import Dataset
import numpy as np
fin = Dataset('f522_dh.trainingdata_in.lcv.20190401_0000z.nc4') 
fout = Dataset('f522_dh.trainingdata_out.lcv.20190401_0000z.nc4')

In [19]:
x = fin.variables
y = fout.variables
for var in x.keys():
    print(var, x[var][:].shape)
    
n   = x['Xdim'].shape[0]    
p   = x['Ydim'].shape[0]

Xdim (720,)
Ydim (4320,)
lons (4320, 720)
lats (4320, 720)
lev (72,)
time (1,)
emis (1, 4320, 720)
fcld (1, 72, 4320, 720)
o3 (1, 72, 4320, 720)
pl (1, 72, 4320, 720)
q (1, 72, 4320, 720)
qi (1, 72, 4320, 720)
ql (1, 72, 4320, 720)
ri (1, 72, 4320, 720)
rl (1, 72, 4320, 720)
t (1, 72, 4320, 720)
ts (1, 4320, 720)


### DATA PREPROCESSING

In [20]:
class Kernel(object):
    def __init__(self, ids):
        self.ids = ids
        
    def apply(self, x):
        return(x)       

class ProdKernel(Kernel):
    def __init__(self, ids=[]):
        Kernel.__init__(self,ids)
        self.ids=ids

    def apply(self,x0, xheader):
        x=x0.copy()
        for ids in self.ids:
            id1=np.where(ids[0]==xheader)[0]
            id2=np.where(ids[1]==xheader)[0]
            k = x[:,:,id1]*x[:,:,id2]
            hx.append(ids[0]+'*'+ids[1])
            x = np.concatenate((k,x),axis=2)
        return(x)
    
Pk = ProdKernel()# [("ts","o3"),("ts","rl") ] )
#x0 = Pk.apply(X, np.array(hx))

In [21]:
print(p,n)

4320 720


In [23]:
def generate(x, y, kernel, batch_size=16):

    n   = x['Xdim'].shape[0]
    p   = x['Ydim'].shape[0]
    lev = x['lev'].shape[0]
    
    xheader = []

    id_batches = np.arange(batch_size)
    while True: 
        
        dataX = np.zeros((batch_size,lev,1))
        idp = id_batches[0] // n
        idn = id_batches % n
        
        while max(id_batches)<n*p:
            
            Y = y['flx'][0,:, idp, idn]
            Y = Y.swapaxes(0,1)
            
            for k in x.keys():
                if(len(x[k].shape)>2):
                    xheader.append(k)
                    if(len(x[k].shape)==4):
                        a = x[k][:,:,idp,idn]
                        a = a.reshape(1,lev, -1)
                        a = a.swapaxes(0,2)
                    elif(len(x[k].shape)==3):
                        a = x[k][:,idp,idn]
                        a = a.repeat(lev,1).reshape(1,-1 , lev)
                        a = a.swapaxes(0,1)
                        a = a.swapaxes(1,2)
                    dataX = np.concatenate((dataX,a),axis=2)
            yield dataX,Y
            id_batches+=batch_size        

from tqdm import tqdm
batch_size=16
generate_batch = lambda A,batch_size : generate(A[0],A[1],A[2],batch_size)
for i in tqdm(range(1)):
    for X,Y in generate_batch( (x,y,Pk),batch_size ):
        print(X.shape,Y.shape)
        break

100%|██████████| 1/1 [00:01<00:00,  1.31s/it]

(16, 72, 12) (16, 73)


CAN BE MODIFY :
   - tride : check
   - padding : change value of padding (same for ex or lin)
   - check act/kernel constraint (limit size of gradient)

In [4]:
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Activation,Input,Flatten
from keras import optimizers, regularizers, initializers

# PARAMETERS 
HNn = [128,64,16,4] # hidden layer size
d=12 # number of variables ############" CHANGE !!
s=0

model = Sequential([
    Conv1D(input_shape=(72,d),filters=HNn[0],kernel_regularizer=regularizers.l2(0.01), \
           kernel_size=10, strides=1, padding="same", kernel_initializer = initializers.Zeros()),
    Activation("relu"),
    Conv1D(filters=HNn[1], kernel_size=10, \
           kernel_regularizer=regularizers.l2(0.01), strides=1, padding="same",kernel_initializer = initializers.RandomUniform(minval=-1, maxval=1, seed=s)),
    Activation("relu"),
    Conv1D(filters=HNn[2], kernel_size=3, \
           kernel_regularizer=regularizers.l2(0.01), strides=1, padding="same",kernel_initializer = initializers.RandomUniform(minval=-1, maxval=1, seed=s)),
    Activation("relu"),
    Conv1D(filters=HNn[3], kernel_size=3, \
           kernel_regularizer=regularizers.l2(0.01), strides=1, padding="same",kernel_initializer = initializers.RandomUniform(minval=-1, maxval=1, seed=s)),
    Activation("relu"),
    Flatten(),
    Dense(73, input_shape=(72*HNn[2],), kernel_regularizer = regularizers.l2(0.01),kernel_initializer = initializers.RandomUniform(minval=-1, maxval=1, seed=s))
])

sgd = optimizers.SGD(lr=0.1, decay=1e-6, momentum=0.9)
model.compile(optimizer=sgd,
             loss='mse',
             )
#model.summary()
model.save('arch1_FCNN.h5')

Using TensorFlow backend.
/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.24.2) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [5]:
# Training :

model.fit_generator( batch_generator( (x, y, K) ,batch_size=16), steps_per_epoch=200, epochs=1)
model.save('arch1_FCNN.h5')

NameError: name 'batch_generator' is not defined

In [ ]:
for layer in model.layers:
    print(layer.get_weights()[0].shape)

In [ ]:
w=model.weights[0]

##### MODEL 2 : FCM-Final FC

##### MODEL 1 : U-net

In [ ]:
##### MODEL 1 : FCM

In [ ]:
# 1 just fc :
# 2 conv :
# 3 read : AvgPool1D, Bayesian NN, Dropout, CNN on the sphere (spherical convolution)

In [8]:
def generate_batch(x, y, batch_size=16):

    n   = x['Xdim'].shape[0]
    p   = x['Ydim'].shape[0]
    lev = x['lev'].shape[0]
    idx = np.random.choice(np.arange(n*p), batch_size, replace=False)
    
    Y = y['flx'][:].reshape(1, lev, -1).swapaxes(0,2)[idx]
    
    databoard = np.zeros((batch_size,lev,1))
    xheader = []

    for k in x.keys():
        a = x[k][:]
        if(len(a.shape)>2):
            xheader.append(k)
            if(len(x[k].shape)==4):
                a = a.reshape(1,lev, -1)
                a = a.swapaxes(0,2)
            elif(len(x[k].shape)==3):
                a = a.reshape(1, -1)
                a = a.repeat(lev,1).reshape(1,-1 , lev)
                a = a.swapaxes(0,1)
                a = a.swapaxes(1,2)
            a = a[idx]
            databoard = np.concatenate((databoard,a),axis=2)
    return(databoard,xheader,Y)

X,hx,Y = generate_batch(x,y)